In [1]:
import numpy as np
import os
import cv2 as cv
import cv2
import matplotlib.pyplot as plt
import math
import glob
from ultralytics import YOLO
from roboflow import Roboflow
import torch
from torchvision.utils import draw_bounding_boxes
from math import sqrt
from math import log
import math

In [2]:
def show_image(image, window_name='image', timeout=0, scale=0.2):
    cv.imshow(window_name, cv.resize(image, None, fx=scale, fy=scale))
    cv.waitKey(timeout)
    cv.destroyAllWindows()

In [3]:
def load_images(image_paths):
    images = [cv.imread(image_path) for image_path in image_paths]
    return images

In [8]:
# TASK 1

# Loading auxiliary and training images; this should not be run because I've included the trained model in the submission archive

auxiliary_images_paths = glob.glob("not used/auxiliary_images/*.jpg")
train_images_paths = glob.glob('not used/train/Task1/*.jpg')
auxiliary_images = load_images(auxiliary_images_paths)
train_images = load_images(train_images_paths)

In [12]:
# image = auxiliary_images[3].copy()
# blurred = cv2.GaussianBlur(image,(11,11),0)
# gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
# edges = cv.Canny(gray, threshold1=30, threshold2=100)

In [9]:
def findEllipses(img):
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    kernel = np.ones((7, 7), np.float32) / 49
    
    blur = cv2.filter2D(img_HSV, -1, kernel)
    h, s, val = cv2.split(blur)
    
    ret, thresh = cv2.threshold(val, 127, 255, cv2.THRESH_BINARY_INV)
    contours, hierarchy = cv2.findContours(thresh, 1, 2)

    # The thresholds for the areas of the ellipses
    min_area_ellipse = 50000
    max_area_ellipse = 5000000
    
    ellipses = []

    for contour in contours:
        try:
            if min_area_ellipse < cv2.contourArea(contour) < max_area_ellipse:
                ellipse = cv2.fitEllipse(contour)
                ellipses.append(ellipse)
        except:
            print ("error")
            continue
    return ellipses, img

In [10]:
def fit_ellipses_to_img(img, ellipses):
    img_copy = img.copy()
    for ellipse in ellipses:
        cv2.ellipse(img_copy, ellipse, (255, 0, 255), 2)
    return img_copy

In [11]:
dartboard_img = auxiliary_images[3].copy()
ellipses, dartboard_img = findEllipses(dartboard_img)

# The center ellipse (the bullseye) is not found with the current implementation, so I will add it with values found empirically
smallest_ellipse = ellipses[0]
dartboard_img_copy = dartboard_img.copy()
center = (smallest_ellipse[0][0] + 12, smallest_ellipse[0][1] - 5)
width = 50 # Found empirically
height = 63 # Found empirically
angle = smallest_ellipse[2]
bullseye_ellipse = (center, (width, height), angle)
ellipses.insert(0, bullseye_ellipse)

dartboard_img = fit_ellipses_to_img(dartboard_img, ellipses)
show_image(dartboard_img, scale=0.4)

In [408]:
# Downloading the annotated dataset
# This section is not necessary to run

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="3oowiNsJPMjPx8J2wNh4")
# project = rf.workspace("dartsdetection-pks4e").project("hard-hat-sample-4zvbg")
# dataset = project.version(10).download("yolov8")

In [409]:
# Loading and training the model from a pretrained yolov8 model
# This section is also not necessary to run because I've included the trained model in the project. It will be loaded later on

# # Load a model
# model = YOLO('yolov8n.pt') 

# # Train the model
# results = model.train(data='ultralytics/ultralytics/cfg/datasets/darts.yaml', epochs=100)

In [12]:
model = YOLO('best_task1.pt')

In [13]:
def draw_predicted_rectangles(img, result):
    img_copy = img.copy()
    boxes = result.boxes.data.tolist()
    
    for box in boxes:
        # Drawing the box:
        top_left = (int(box[0]), int(box[1]))
        bottom_right = (int(box[2]), int(box[3]))
        color = (0, 255, 0)
        thickness = 5
        img_copy = cv.rectangle(img_copy, top_left, bottom_right, color, thickness)

        # Writing the annotation:
        annotation = result.names[box[-1]]
        position = (50, 50)
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 3
        font_color = (0, 0, 255)
        font_thickness = 5
        
        cv2.putText(img_copy, annotation, bottom_right, font, font_scale, font_color, font_thickness)

    return img_copy

def find_darts(model, imgs, show_results=False):
    results = model.predict(imgs)
    if show_results:
        for i, result in enumerate(results):
            img_copy = imgs[i].copy()
            img_copy = draw_predicted_rectangles(img_copy, result)
            show_image(img_copy)
    return results

In [14]:
val_images_paths = glob.glob('not used/evaluation/fake_test/Task1/*.jpg')
val_images = load_images(val_images_paths)

In [15]:
results = find_darts(model, val_images, False)
#results = model.predict(train_images)


0: 640x480 3 darts, 1: 640x480 2 darts, 2: 640x480 3 darts, 3: 640x480 2 darts, 4: 640x480 1 dart, 1 dart_tip, 5: 640x480 2 darts, 1 dart_tip, 6: 640x480 1 dart, 7: 640x480 1 dart, 8: 640x480 2 darts, 9: 640x480 2 darts, 10: 640x480 3 darts, 11: 640x480 1 dart, 12: 640x480 3 darts, 2 dart_tips, 13: 640x480 3 darts, 1 dart_tip, 14: 640x480 2 darts, 1 dart_tip, 15: 640x480 1 dart, 16: 640x480 3 darts, 2 dart_tips, 17: 640x480 3 darts, 1 dart_tip, 18: 640x480 1 dart, 19: 640x480 3 darts, 1 dart_tip, 20: 640x480 1 dart, 21: 640x480 2 darts, 1 dart_tip, 22: 640x480 2 darts, 23: 640x480 2 darts, 24: 640x480 1 dart, 1 dart_tip, 3200.7ms
Speed: 5.4ms preprocess, 128.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)


In [16]:
def is_point_inside_ellipse(point, ellipse):
    rect_points = cv2.boxPoints(ellipse).astype(np.intp)
    point_inside_rectangle = cv2.pointPolygonTest(rect_points, point, False) >= 0
    return point_inside_rectangle

In [17]:
def solve_images(model, images, ellipses, show_darts, show_results):
    results = find_darts(model, images, show_darts)
    final_scores = []
    for i, result in enumerate(results):
        img = images[i].copy()
        boxes = result.boxes.data.tolist()
        dart_boxes = [box for box in boxes if box[-1]==0] #box[-1]==1 corresponds with dart_tips, which we don't need
        nr_darts = len(dart_boxes)
        print(f"{nr_darts} darts:")
        dart_scores = []
        for box in dart_boxes:
            x_dart = int(box[0])
            y_dart = int((box[1] + box[3])/2)
            dart_point = (x_dart, y_dart)
            for j, ellipse in enumerate(ellipses):
                if is_point_inside_ellipse(dart_point, ellipse):
                    point_color = (0, 255, 0)
                    dart_score = 10-j
                    dart_scores.append(dart_score)
                    cv2.ellipse(img, ellipse, (0, 0, 255), 2)
                    break
                else:
                    point_color = (0, 0, 255)
            point_radius = 7
            cv2.circle(img, dart_point, point_radius, point_color, -1)
        dart_scores.sort()
        for k, score in enumerate(dart_scores):
            print(f"dart {k} has score {score}")
        final_score = (nr_darts, dart_scores)
        final_scores.append(final_score)
        if show_results:
            show_image(img, scale=0.3)
    return final_scores

In [18]:
test_image_paths = glob.glob('test/Task1/*.jpg') # !!!!!!!!!!!!!!! CHANGE BEFORE SUBMISSION !!!!!!!!!!!!!!!!!!!!!!!
test_images = load_images(test_image_paths)
final_scores = solve_images(model, test_images, ellipses, False, False)
destination_folder = 'Radu_Madalin_407/Task1/'
image_names = [path.split('\\')[-1].split('.')[0] + "_predicted" for path in test_image_paths]
for i, final_score in enumerate(final_scores):
    nr_darts, scores = final_score
    file_name = image_names[i] + ".txt"
    with open(destination_folder + '/' + file_name, 'w') as file:
        file.write(f"{nr_darts}\n")
        for score in scores:
            file.write(f"{score}\n")


0: 640x480 2 darts, 2 dart_tips, 1: 640x480 3 darts, 2 dart_tips, 2: 640x480 1 dart, 3: 640x480 2 darts, 1 dart_tip, 4: 640x480 3 darts, 1 dart_tip, 5: 640x480 2 darts, 6: 640x480 3 darts, 1 dart_tip, 7: 640x480 3 darts, 8: 640x480 1 dart, 1 dart_tip, 9: 640x480 3 darts, 10: 640x480 2 darts, 2 dart_tips, 11: 640x480 1 dart, 12: 640x480 2 darts, 13: 640x480 1 dart, 1 dart_tip, 14: 640x480 2 darts, 15: 640x480 2 darts, 16: 640x480 2 darts, 1 dart_tip, 17: 640x480 1 dart, 18: 640x480 2 darts, 19: 640x480 3 darts, 2 dart_tips, 20: 640x480 2 darts, 21: 640x480 2 darts, 1 dart_tip, 22: 640x480 2 darts, 1 dart_tip, 23: 640x480 1 dart, 1 dart_tip, 24: 640x480 2 darts, 2918.5ms
Speed: 4.6ms preprocess, 116.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 480)


2 darts:
dart 0 has score 4
dart 1 has score 5
3 darts:
dart 0 has score 5
dart 1 has score 6
dart 2 has score 8
1 darts:
dart 0 has score 9
2 darts:
dart 0 has score 6
dart 1 has score 8
3 darts:
dart 0 has score 6
dart 1 has score 7
dart 2 has score 10
2 darts:
dart 0 has score 5
dart 1 has score 6
3 darts:
dart 0 has score 3
dart 1 has score 6
dart 2 has score 8
3 darts:
dart 0 has score 5
dart 1 has score 5
dart 2 has score 6
1 darts:
dart 0 has score 4
3 darts:
dart 0 has score 4
dart 1 has score 7
dart 2 has score 9
2 darts:
dart 0 has score 6
dart 1 has score 7
1 darts:
dart 0 has score 6
2 darts:
dart 0 has score 6
dart 1 has score 8
1 darts:
dart 0 has score 9
2 darts:
dart 0 has score 6
dart 1 has score 7
2 darts:
dart 0 has score 5
dart 1 has score 6
2 darts:
dart 0 has score 6
dart 1 has score 9
1 darts:
dart 0 has score 8
2 darts:
dart 0 has score 5
dart 1 has score 7
3 darts:
dart 0 has score 5
dart 1 has score 6
dart 2 has score 8
2 darts:
dart 0 has score 6
dart 1 has s

In [19]:
# TASK 2

train_images_paths = glob.glob('not used/train/Task2/*.jpg')
train_images = load_images(train_images_paths)

In [21]:
def extract_color_points(img, lower1, upper1, lower2, upper2, show_result=False):
    image_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(image_HSV, lower1, upper1)
    mask2 = cv2.inRange(image_HSV, lower2, upper2)
    mask = cv2.bitwise_or(mask1, mask2)
    
    result = cv2.bitwise_and(img, dartboard_img, mask=mask)
    if show_result:
        show_image(result)

    points = np.column_stack(np.where(mask > 0))
    points = points[:, [1, 0]]
    return mask, points

In [22]:
dartboard_img = auxiliary_images[-1].copy()
#blur = cv2.GaussianBlur(dartboard_img, (5, 5), 0)

lower_green1 = np.array([35, 50, 50])
upper_green1 = np.array([80, 255, 255])
lower_green2 = np.array([80, 50, 50])
upper_green2 = np.array([100, 255, 255])

green_mask, green_points = extract_color_points(dartboard_img, lower_green1, upper_green1, lower_green2, upper_green2)

lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

red_mask, red_points = extract_color_points(dartboard_img, lower_red1, upper_red1, lower_red2, upper_red2)

red_and_green = cv2.bitwise_or(red_mask, green_mask)

lower_white = (150, 150, 150)
upper_white = (255, 255, 255)

white_mask = cv2.inRange(dartboard_img, lower_white, upper_white)
white_points = np.column_stack(np.where(white_mask > 0))
white_points = white_points[:, [1, 0]]

lower_black = (0, 0, 0)
upper_black = (70, 70, 70)

black_mask = cv2.inRange(dartboard_img, lower_black, upper_black)
black_points = np.column_stack(np.where(black_mask > 0))
black_points = black_points[:, [1, 0]]

In [217]:
show_image(black_mask)
show_image(white_mask)

In [420]:
# show_image(green_mask)
# order -> red, green, white, black

In [23]:
def find_ellipses2(img, thresh, minArea=3000, maxArea=5000000, show_results=False):
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    ellipses = []
    for contour in contours:
        try:
            if minArea < cv2.contourArea(contour) < maxArea:
                ellipse = cv2.fitEllipse(contour)
                center = ellipse[0]
                width, height = ellipse[1]
                angle = ellipse[2]
                if (width/height * 100) > 60:
                    # print(cv2.contourArea(contour), width/height * 100)
                    ellipses.append(ellipse)
        except:
                print ("error")
                continue
    
    img = fit_ellipses_to_img(img, ellipses)
    if show_results:
        show_image(img, scale=0.3)

    return ellipses, img

In [24]:
dartboard_img = auxiliary_images[-1].copy()
dartboard_ellipses, dartboard_img = find_ellipses2(dartboard_img, black_mask, minArea=1000000, maxArea=2000000)
#show_image(dartboard_img, scale=0.4)
assert(len(dartboard_ellipses) == 1)
dartboard_ellipse = dartboard_ellipses[0]
dartboard_center = (int(dartboard_ellipse[0][0]), int(dartboard_ellipse[0][1]))
dartboard_width, dartboard_height = dartboard_ellipse[1]
dartboard_angle = dartboard_ellipse[2]

In [25]:
dartboard_img = auxiliary_images[-1].copy()
red_center_ellipses, dartboard_img = find_ellipses2(dartboard_img, red_mask, minArea=200, maxArea=5000, show_results=False)
assert(len(red_center_ellipses)==1)
center_ellipse = red_center_ellipses[0]
center = (int(center_ellipse[0][0]), int(center_ellipse[0][1]))
center_width, center_height = center_ellipse[1]
center_angle = center_ellipse[2]
#show_image(dartboard_img)

In [26]:
dartboard_img = auxiliary_images[-1].copy()
green_center_ellipses, dartboard_img = find_ellipses2(dartboard_img, green_mask, minArea=5000, maxArea=50000, show_results=False)
assert(len(green_center_ellipses)==1)
green_center_ellipse = green_center_ellipses[0]
green_center = (int(green_center_ellipse[0][0]), int(green_center_ellipse[0][1]))
green_center_width, green_center_height = green_center_ellipse[1]
green_center_angle = green_center_ellipse[2]
print(center_angle)

174.33786010742188


In [425]:
# show_image(white_mask)
# show_image(black_mask)
# show_image(green_mask)
# show_image(red_mask)

In [10]:
# # Loading and training the model from a pretrained yolov8 model
# # This section is also not necessary to run because I've included the trained model in the project. It will be loaded later on

# # Load a model
# model = YOLO('yolov8n.pt') 

# # Train the model
# results = model.train(data='ultralytics/ultralytics/cfg/datasets/darts2.yaml', epochs=100)

New https://pypi.org/project/ultralytics/8.0.168 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.157  Python-3.11.4 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i5-11400H 2.70GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=ultralytics/ultralytics/cfg/datasets/darts2.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, 

In [27]:
model = YOLO('best_task2.pt')

In [28]:
val_images_paths = glob.glob('not used/evaluation/fake_test/Task2/*.jpg')
val_images = load_images(val_images_paths)

In [29]:
results = find_darts(model, val_images[:5], True)


0: 640x480 2 darts, 1: 640x480 1 dart, 2: 640x480 2 darts, 3: 640x480 2 darts, 4: 640x480 3 darts, 451.0ms
Speed: 4.2ms preprocess, 90.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 480)


In [30]:
reds_list = red_points.tolist()
greens_list = green_points.tolist()
white_list = white_points.tolist()
black_list = black_points.tolist()

In [31]:
sector_angles = list(range(9, 360, 18))
sector_to_points_dict = {
    0:6,
    1:10,
    2:15,
    3:2,
    4:17,
    5:3,
    6:19,
    7:7,
    8:16,
    9:8,
    10:11,
    11:14,
    12:9,
    13:12,
    14:5,
    15:20,
    16:1,
    17:18,
    18:4,
    19:13
}

def angle_to_points(angle):
    sector = int(((angle+9)%360)/18)
    return sector_to_points_dict[sector]

# point_x, point_y =  dartboard_center
# sector_degrees = 360/20

# center_x, center_y = center

def get_point_score(point, img):
    center_x, center_y = center
    point_x, point_y =  point
    relative_x = point_x - center_x
    relative_y = point_y - center_y
    
    angle_rad = np.radians(-dartboard_angle)
    
    rotated_x = relative_x * np.cos(angle_rad) - relative_y * np.sin(angle_rad)
    rotated_y = relative_x * np.sin(angle_rad) + relative_y * np.cos(angle_rad)
    
    angle_point = np.degrees(np.arctan2(rotated_y, rotated_x))

    offset = 4.65
    #offset = 0
    desired_angle_degrees = angle_point + offset
    desired_angle_radians = math.radians(desired_angle_degrees)
    
    endpoint_x = center_x + dartboard_width * math.cos(desired_angle_radians)
    endpoint_y = center_y + dartboard_height * math.sin(desired_angle_radians)
    image = img.copy()
    image = cv2.line(image, (int(center_x), int(center_y)), (int(endpoint_x), int(endpoint_y)), (255, 0, 0), 5)
    #show_image(dartboard_img, scale=0.3)

    return angle_to_points(desired_angle_degrees), image

#get_point_score((dartboard_center[0], dartboard_center[1]+1))

In [32]:
def euclidian_distance(point1, point2):
    return sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

In [33]:
def get_point_flag(green_ellipse, red_ellipse, point):
    print(f"Point {dart_point} in reds: {dart_point_list in reds_list}")
    print(f"Point {dart_point} in greens: {dart_point_list in greens_list}")
    print(f"Point {dart_point} in whites: {dart_point_list in white_list}")
    print(f"Point {dart_point} in blacks: {dart_point_list in black_list}")
    return flag

def solve_images2(model, images, show_darts, show_results):
    results = find_darts(model, images, show_darts)
    final_scores = []
    for i, result in enumerate(results):
        img = images[i].copy()
        boxes = result.boxes.data.tolist()
        dart_boxes = [box for box in boxes if box[-1]==0]
        nr_darts = len(dart_boxes)
        print(f"{nr_darts} darts:")
        dart_scores = []
        dart_scores.append(nr_darts)
        for box in dart_boxes:
            x_dart = int(box[0])
            y_dart = int((box[1] + box[3])/2)

            correction_x = 8
            standard_height = 219
            standard_ratio = 0.4
            box_width = box[2]-box[0]
            box_height = box[3]-box[1]
            actual_ratio = box_height/box_width
            # correction_y = int(log(sqrt((actual_ratio - standard_ratio) ** 2)))
            correction_y = int(((actual_ratio)**4) * 1000)
            # correction_y = int(((int(box_height)/standard_height - 1) ** 2) * 10000)

            cv2.circle(img, (x_dart, y_dart), 7, (255,0,255), -1)
            
            dart_point = (x_dart + correction_x, y_dart + correction_y)
            dart_point_list = list(dart_point)

            point_color = (0, 255, 0)
            point_radius = 7
            cv2.circle(img, dart_point, point_radius, point_color, -1)

            if is_point_inside_ellipse(dart_point, green_center_ellipse):
                if dart_point_list in reds_list or is_point_inside_ellipse(dart_point, center_ellipse):
                    score = "b50"
                    dart_scores.append(score)
                    continue
                else:
                    score = "b25"
                    dart_scores.append(score)
                    continue
                    
            nr_score, img = get_point_score(dart_point, img)
            flag = "s"
            if dart_point_list in reds_list or dart_point_list in greens_list:
                if euclidian_distance(center, dart_point) > 600:
                    flag = "d"
                else:
                    flag = "t"
            score = flag + str(nr_score)
            print(score)
            dart_scores.append(score)

        final_scores.append(dart_scores)
        # dart_scores.sort()
        # for k, score in enumerate(dart_scores):
        #     print(f"dart {k} has score {score}")
        # final_score = (nr_darts, dart_scores)
        # final_scores.append(final_score)
        if show_results:
            show_image(img, scale=0.3)
    return final_scores


In [435]:
# final_scores = solve_images2(model, val_images, False, True)

In [34]:
test_image_paths = glob.glob('test/Task2/*.jpg') # !!!!!!!!!!!!!!! CHANGE BEFORE SUBMISSION !!!!!!!!!!!!!!!!!!!!!!!
#test_image_paths = glob.glob('train/Task2/*.jpg') # !!!!!!!!!!!!!!! CHANGE BEFORE SUBMISSION !!!!!!!!!!!!!!!!!!!!!!!
test_images = load_images(test_image_paths)
final_scores = solve_images2(model, test_images, False, False)
destination_folder = 'Radu_Madalin_407/Task2/'
image_names = [path.split('\\')[-1].split('.')[0] + "_predicted" for path in test_image_paths]
for i, final_score in enumerate(final_scores):
    file_name = image_names[i] + ".txt"
    with open(destination_folder + '/' + file_name, 'w') as file:
        for line in final_score:
            file.write(str(line) + "\n") 


0: 640x480 2 darts, 1: 640x480 1 dart, 2: 640x480 2 darts, 3: 640x480 3 darts, 4: 640x480 2 darts, 5: 640x480 2 darts, 6: 640x480 2 darts, 7: 640x480 1 dart, 8: 640x480 2 darts, 9: 640x480 4 darts, 10: 640x480 3 darts, 11: 640x480 1 dart, 12: 640x480 1 dart, 13: 640x480 2 darts, 14: 640x480 3 darts, 15: 640x480 2 darts, 16: 640x480 1 dart, 17: 640x480 2 darts, 18: 640x480 2 darts, 19: 640x480 2 darts, 20: 640x480 3 darts, 21: 640x480 2 darts, 22: 640x480 1 dart, 23: 640x480 3 darts, 24: 640x480 2 darts, 3282.3ms
Speed: 6.4ms preprocess, 131.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)


2 darts:
s10
s9
1 darts:
s20
2 darts:
s1
s17
3 darts:
s14
s1
s10
2 darts:
s1
s11
2 darts:
s18
t11
2 darts:
s15
s11
1 darts:
s12
2 darts:
s13
s12
4 darts:
s7
s13
s12
s5
3 darts:
s18
s12
s5
1 darts:
s1
1 darts:
t11
2 darts:
s18
s12
3 darts:
s4
t19
t14
2 darts:
s19
d5
1 darts:
s13
2 darts:
s15
s1
2 darts:
s19
s20
2 darts:
s4
s1
3 darts:
s14
s1
s10
2 darts:
s18
s13
1 darts:
s17
3 darts:
t2
t13
s5
2 darts:
t19
s1
